### Set Up for Project Imports


In [25]:
import sys
from pathlib import Path

In [26]:
# Add repo root (one level up from 'source/')
ROOT = Path.cwd().parent
if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))

In [27]:
# Auto-reload code changes
%load_ext autoreload
%autoreload 2

## Imports

In [78]:
from data.api import UcIrvineAPI, BureauEconomicAnalysisAPI
from data.dataset_id import DatasetIDs
import json
import pandas

In [29]:
pandas.set_option('display.max_colwidth', None) # show all text in cells

## Data

In [79]:
apartment_for_rent_classified = UcIrvineAPI.fetch_dataset(repo_id= DatasetIDs.Apartment_For_Rent_Classified.value)

/Users/isaacpacheco/Documents/School/Graduate/Fall 2025/CS 577 - Data Science/Project/Github/.venv/lib/python3.13/site-packages/ucimlrepo/fetch.py:97: DtypeWarning: Columns (0,5,6,12,14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data_url)


In [31]:
print(json.dumps(apartment_for_rent_classified.metadata, indent=2))

{
  "uci_id": 555,
  "name": "Apartment for Rent Classified",
  "repository_url": "https://archive.ics.uci.edu/dataset/555/apartment+for+rent+classified",
  "data_url": "https://archive.ics.uci.edu/static/public/555/data.csv",
  "abstract": "This is a dataset of classified for apartments for rent in USA.\n",
  "area": "Business",
  "tasks": [
    "Classification",
    "Regression",
    "Clustering"
  ],
  "characteristics": [
    "Multivariate"
  ],
  "num_instances": 10000,
  "num_features": 21,
  "feature_types": [
    "Categorical",
    "Integer"
  ],
  "demographics": [],
  "target_col": null,
  "index_col": [
    "id"
  ],
  "has_missing_values": "no",
  "missing_values_symbol": null,
  "year_of_dataset_creation": 2019,
  "last_updated": "Mon Feb 26 2024",
  "dataset_doi": "10.24432/C5X623",
  "creators": [],
  "intro_paper": null,
  "additional_info": {
    "summary": "The dataset contains of 10'000 or 100'000 rows and of 22 columns The data has been cleaned in the way that \r\nc

In [82]:
data = BureauEconomicAnalysisAPI.fetch_dataset()
print(data)

       Code GeoFips               GeoName TimePeriod  \
0   SARPP-1   00000         United States       2019   
1   SARPP-1   01000               Alabama       2019   
2   SARPP-1   02000                Alaska       2019   
3   SARPP-1   04000               Arizona       2019   
4   SARPP-1   05000              Arkansas       2019   
5   SARPP-1   06000            California       2019   
6   SARPP-1   08000              Colorado       2019   
7   SARPP-1   09000           Connecticut       2019   
8   SARPP-1   10000              Delaware       2019   
9   SARPP-1   11000  District of Columbia       2019   
10  SARPP-1   12000               Florida       2019   
11  SARPP-1   13000               Georgia       2019   
12  SARPP-1   15000                Hawaii       2019   
13  SARPP-1   16000                 Idaho       2019   
14  SARPP-1   17000              Illinois       2019   
15  SARPP-1   18000               Indiana       2019   
16  SARPP-1   19000                  Iowa       